In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

# Pruebo sin usar datos de los negocios

In [6]:
set_pruebas = pd.read_csv("../properati_dataset_modificado2.csv")
set_pruebas.loc[:,'property_type'].value_counts()

apartment    9761
house        3409
PH            996
Name: property_type, dtype: int64

In [7]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min)\
                              & (propiedades.property_type != 'store'),:]

set_pruebas = pd.read_csv("../properati_dataset_modificado2.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [9]:
cant = 15
col = ['superficie','lat', 'lon','property_type_encoded']
comodidades = ['seguridad','gimnasio','aire','pileta','cochera','transporte','comercio','servicios','doble piso','jardin']

for i in range(cant):
    com = list(np.random.choice(comodidades,6,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[:,columnas]
    set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']
    print(i,end = '-')

0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-

In [10]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_bagging_definitivo_sinstores.csv', index = False)

# Pruebo separando los datos de Capital por un lado, y los de GBA por el otro

In [2]:
propiedades = pd.read_csv('../../solo_capital_modificado.csv')
set_pruebas = pd.read_csv("../properati_dataset_modificado2.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [3]:
columnas = ['lat', 'lon','superficie','property_type_encoded','distancia_subtes', \
       'distancia_monumentos', 'Zona_inundacion', 'distancia_estadios', 'distancia_villas']

In [4]:
set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)

set_pruebas.loc[set_pruebas.capital == True,'price_usd'] = \
                set_pruebas.loc[set_pruebas.capital == True,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)

### Para los que no son capital

In [5]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min)\
                              & (propiedades.state_name != 'Capital Federal'),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [6]:
tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)

set_pruebas.loc[set_pruebas.capital == False,'price_usd'] = \
                set_pruebas.loc[set_pruebas.capital == False,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)

### Resultado final

In [7]:
resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_separando_capital.csv', index = False)
resultado.describe()

,id,price_usd
count,1.416600e+04,1.416600e+04
mean,2.285382e+06,2.316872e+05
std,4.290968e+04,2.427909e+05
min,3.632000e+03,3.000000e+04
25%,2.271809e+06,9.800000e+04
50%,2.290284e+06,1.580000e+05
75%,2.309395e+06,2.675000e+05
max,2.332338e+06,2.400000e+06


# Ahora los que no son Capital los agrupo por Tipo de propiedad

In [8]:
propiedades = pd.read_csv('../../solo_capital_modificado.csv')
set_pruebas = pd.read_csv("../properati_dataset_modificado2.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [9]:
columnas = ['lat', 'lon','superficie','property_type_encoded','distancia_subtes', \
       'distancia_monumentos', 'Zona_inundacion', 'distancia_estadios', 'distancia_villas']

In [10]:
set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)

set_pruebas.loc[set_pruebas.capital == True,'price_usd'] = \
                set_pruebas.loc[set_pruebas.capital == True,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)

### Para los que no son capital

In [11]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min)\
                              & (propiedades.state_name != 'Capital Federal'),:]

columnas = ['superficie','lat', 'lon','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin']

In [12]:
tipos = ['PH','apartment','house']

for tipo in tipos:
    
    set_entrenamiento_datos = propiedades.loc[propiedades.property_type == tipo,columnas]
    set_entrenamiento_resultado = propiedades.loc[propiedades.property_type == tipo,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)

    set_pruebas.loc[(set_pruebas.capital == False) & (set_pruebas.property_type == tipo),'price_usd'] = \
                    set_pruebas.loc[(set_pruebas.capital == False) & (set_pruebas.property_type == tipo),columnas]\
                    .apply(lambda x: tree.predict([x])[0],axis = 1)

### Resultado final

In [13]:
resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_separando_capital_y_tipos.csv', index = False)
resultado.describe()

,id,price_usd
count,1.416600e+04,1.416600e+04
mean,2.285382e+06,2.288785e+05
std,4.290968e+04,2.189198e+05
min,3.632000e+03,3.000000e+04
25%,2.271809e+06,9.900000e+04
50%,2.290284e+06,1.588500e+05
75%,2.309395e+06,2.700000e+05
max,2.332338e+06,2.500000e+06
